In [ ]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import load_model

In [161]:
TIMEFRAMES = [7, 14, 30]
PREDICTIONS = [1, 5]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

# Select whether to use transfer learning or not
TRANSFER = True
if TRANSFER:
    labels = pd.read_csv('labels/transfer_labels.csv')
else:
    labels = pd.read_csv('labels/test_labels.csv')

In [ ]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 7:
            # Crop the image using the bounding rectangle
            crop = img[105:105+115, 80:80+38]
        elif number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [162]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_53465/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [164]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [165]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

0    4832
1    3376
Name: Label, dtype: int64


In [163]:
# Sort the DataFrame by date
# Extract only data that includes Treasury in the filename
labels = labels[labels['Image'].str.contains('Silver')]
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                   Image  TimePrediction  \
8208   images/MPA/Silver_2023-01-01 00:00:00_7_croppe...               1   
13791  images/API/Silver_2023-01-01 00:00:00_30_cropp...               5   
13790  images/MLC/Silver_2023-01-01 00:00:00_30_cropp...               5   
13789  images/CPA/Silver_2023-01-01 00:00:00_30_cropp...               5   
13788  images/MPA/Silver_2023-01-01 00:00:00_30_cropp...               5   

       LastPrice  FuturePrice  Label       Date  
8208       21.74        21.74      0 2023-01-01  
13791      21.83        20.48      0 2023-01-01  
13790      21.83        20.48      0 2023-01-01  
13789      21.83        20.48      0 2023-01-01  
13788      21.83        20.48      0 2023-01-01  


In [ ]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_investment += 100
    
    if long_investment > 0:
        # Calculate average long RoR
        long_avg_ror = np.mean(long_ror)
    else:
        long_avg_ror = 0
    
    if short_investment > 0:
        # Calculate average short RoR
        short_avg_ror = np.mean(short_ror)
    else:
        short_avg_ror = 0

    return long_avg_ror, long_investment, short_avg_ror, short_investment

In [ ]:
def get_actual_returns(y_pred_binary, lastPrice, futurePrice, prediction):
    y_pred_binary = y_pred_binary.tolist()

    # Create a list that will store the investment as long as it is locked due to the prediction window
    long_investment = [0] * prediction
    short_investment = [0] * prediction
    both_investment = [0] * prediction

    counter = 0
    for i in range(len(y_pred_binary)):
        if counter == prediction:
            counter = 0
            
        # Long position
        if y_pred_binary[i] == 1:
            long_return = ((futurePrice[i] - lastPrice[i])/lastPrice[i]) + 1
            long_investment[counter] = (long_investment[counter]+100)*(long_return)
            both_investment[counter] = (both_investment[counter]+100)*(long_return)
            counter += 1

        # Short position
        else:
            short_return = ((lastPrice[i] - futurePrice[i])/lastPrice[i]) + 1
            short_investment[counter] = (short_investment[counter]+100)*(short_return)
            both_investment[counter] = (both_investment[counter]+100)*(short_return)
            counter += 1

    # Sum all long trades
    long_returns = np.sum(long_investment)
    # Sum all short trades
    short_returns = np.sum(short_investment)
    # Sum all trades
    both_returns = np.sum(both_investment)

    return long_returns, short_returns, both_returns

In [168]:
# Get baseline RoR for each image type, timeframe, and prediction

ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Investment', 'Return', 'RoR'])

if TRANSFER:
    etf_names = ["Silver"]

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]                
                data = data.reset_index(drop=True)

                investment = 0
                returns = 0

                for etf in etf_names:
                    # Filter the data
                    etf_data = data[data['Image'].str.contains(f'{etf}')]

                
                    lastPrice = etf_data['LastPrice']
                    # Futur price is the last price in the dataset at which all ETFs will be sold
                    futurePrice = etf_data['FuturePrice'].iloc[-1]

                    # Calculate Rate of Return RoR per trade
                    ror = np.array((futurePrice - lastPrice)/lastPrice)

                    # Number of trades
                    investment += len(ror)*100

                    # Sum all trades
                    returns += np.sum((1+ror)*100)
                    # Calculate average RoR

                total_ror = returns/investment

                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Investment': investment,
                    'Return': returns,
                    'RoR': total_ror
                }, ignore_index=True)

if TRANSFER:
    ror_df.to_csv('Silver_baseline_ror.csv', index=False)
else:
    ror_df.to_csv('test_baseline_ror.csv', index=False)

In [ ]:
# Evaluate the model on the test and transfer data without rolling window 

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Returns', 'Short_Average_RoR', 'Short_Investment', 'Short_Returns', 'Total_Investment', 'Total_Returns'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Load the model
                model_filename = f"models/MSCIWorld_{img_type}_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)

                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Evaluate the model on test data
                y_pred = model.predict(X)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)  
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)
                # add true negative rate
                

                y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                long_avg_ror, long_investment, short_avg_ror, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
                # Total investment
                total_investment = long_investment + short_investment

                long_returns, short_returns, both_returns = get_actual_returns(y_pred_binary, lastPrice, futurePrice, prediction)

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Returns' : long_returns,
                    'Long_RoR': long_returns/long_investment,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment, 
                    'Short_Returns' : short_returns,
                    'Short_RoR': short_returns/short_investment,
                    'Total_Investment': total_investment,
                    'Total_Returns': both_returns,
                    'Total_RoR': both_returns/total_investment
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/separate/transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/separate/MSCI_test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/MSCI_test_evaluation_scores.csv'.")

In [ ]:
# Overlay our three new images and evaluate the combined models on the newly crated dataset

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 
                                      'Long_Average_RoR', 'Long_Investment', 'Long_Returns', 'Long_RoR', 
                                      'Short_Average_RoR', 'Short_Investment', 'Short_Returns', 'Short_RoR', 
                                      'Total_Investment', 'Total_Returns', 'Total_RoR'])

TIMEFRAMES = [7, 14, 30]
PREDICTIONS = [1, 5]

IMG_TYPES = ['CPA', 'MLC', 'API']

if TRANSFER:
    etf_names = ["SmallCap", "Treasury", "Silver", "Semiconductor"]
else:
    etf_names = ["MSCIWorld"]

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Evaluating model predicting {prediction} days ahead using overelayed images with {timeframe} days timeframe.")

            # Filter the data
            data = labels[(labels['TimePrediction'] == prediction) &
                            (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)

            # Build df with overlayed images
            overlayed_df = pd.DataFrame(columns=['Image_Array', 'LastPrice', 'FuturePrice', 'Label'])

            # For each timeframe, prediction and ETF overlay the images
            for date in data['Date'].unique():
                for etf in etf_names:
                    try:
                        # Filter the data
                        data_date = data[(data['Date'] == date) & (data['Image'].str.contains(f'{etf}'))]
                        # Load the images
                        img1 = data_date[data_date['Image'].str.contains('CPA')]['Image_Array'].values[0]
                        img2 = data_date[data_date['Image'].str.contains('MLC')]['Image_Array'].values[0]
                        img3 = data_date[data_date['Image'].str.contains('API')]['Image_Array'].values[0]
                        # Overlay the images
                        img = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)
                        img = cv2.addWeighted(img, 0.5, img3, 0.5, 0)

                        # Save image to file
                        cv2.imwrite(f'images/overlayed/{etf}_{date}_{prediction}_{timeframe}.png', img)

                        # Add the image to the DataFrame
                        overlayed_df = overlayed_df.append({
                            'Image_Array': img,
                            'LastPrice': data_date['LastPrice'].values[0],
                            'FuturePrice': data_date['FuturePrice'].values[0],
                            'Label': data_date['Label'].values[0]
                        }, ignore_index=True)
                    except:
                        pass
            overlayed_df = overlayed_df.reset_index(drop=True)

            # Load the model
            model_filename = f"models/new_combined_{timeframe}_{prediction}.h5"
            model = load_model(model_filename)

            X = np.array(overlayed_df['Image_Array'].tolist()) / 255.0
            lastPrice = overlayed_df['LastPrice'].tolist()
            futurePrice = overlayed_df['FuturePrice'].tolist()
            y = overlayed_df['Label'].values

            # Evaluate the model on test data
            y_pred = model.predict(X)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)  

            y_list = list(y)
            y_pred_binary_list = list(y_pred_binary)

            y = np.array(y_list)
            y_pred_binary = np.array(y_pred_binary_list)       

            
            accuracy = accuracy_score(y, y_pred_binary)
            precision = precision_score(y, y_pred_binary)
            recall = recall_score(y, y_pred_binary)
            f1_score = fbeta_score(y, y_pred_binary, beta=1)

            y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            long_avg_ror, long_investment, short_avg_ror, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
            # Total investment
            total_investment = long_investment + short_investment

            long_returns, short_returns = get_actual_returns(y_pred_binary, lastPrice, futurePrice, prediction)
            total_returns = long_returns + short_returns

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Long_Average_RoR': long_avg_ror,
                'Long_Investment': long_investment,
                'Long_Returns' : long_returns,
                'Long_RoR': long_returns/long_investment,
                'Short_Average_RoR': short_avg_ror,
                'Short_Investment': short_investment, 
                'Short_Returns' : short_returns,
                'Short_RoR': short_returns/short_investment,
                'Total_Investment': total_investment,
                'Total_Returns': total_returns,
                'Total_RoR': total_returns/total_investment
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/overlayed/transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/overlayed/MSCIWorld_test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/test_evaluation_scores.csv'.")

In [166]:
# Separate the data into 12 windows and evaluate the model on each window

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 
                                      'Long_Average_RoR', 'Long_Investment', 'Long_Returns', 'Long_RoR', 
                                      'Short_Average_RoR', 'Short_Investment', 'Short_Returns', 'Short_RoR', 
                                      'Total_Investment', 'Total_Returns', 'Total_RoR'])

TIMEFRAMES = [7, 14, 30]
PREDICTIONS = [1, 5]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Load the model
                model_filename = f"models/MSCIWorld_{img_type}_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)
                

                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values
                
                # Initialize y_pred_binary as an empty array
                y_pred_binary = np.array([], dtype=int)
                
                # Create a rolling window of 10
                window_size = len(X)/12
                print(int(window_size))

                for window in range(12):
                    if window == 0:
                        window_X = X[0:int(window_size)]
                        window_lastPrice = lastPrice[0:int(window_size)]
                        window_futurePrice = futurePrice[0:int(window_size)]
                        window_y = y[0:int(window_size)]

                        # Evaluate the model on test data
                        y_pred_window = model.predict(window_X)
                        # Convert predictions to binary: if > 0.5 then 1 else 0
                        y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                        y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])
                    elif window == 11:
                        model.fit(window_X, window_y, epochs=10, batch_size=32, validation_split=0.2)
                        window_X = X[int(window_size*window):]
                        window_lastPrice = lastPrice[int(window_size*window):]
                        window_futurePrice = futurePrice[int(window_size*window):]
                        window_y = y[int(window_size*window):]

                        # Evaluate the model on test data
                        y_pred_window = model.predict(window_X)
                        # Convert predictions to binary: if > 0.5 then 1 else 0
                        y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                        y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])
                    else:
                        model.fit(window_X, window_y, epochs=10, batch_size=32, validation_split=0.2)
                        window_X = X[int(window_size*window):int(window_size*(window+1))]
                        window_lastPrice = lastPrice[int(window_size*window):int(window_size*(window+1))]
                        window_futurePrice = futurePrice[int(window_size*window):int(window_size*(window+1))]
                        window_y = y[int(window_size*window):int(window_size*(window+1))]

                        # Evaluate the model on test data
                        y_pred_window = model.predict(window_X)
                        # Convert predictions to binary: if > 0.5 then 1 else 0
                        y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                        y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)

                y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                long_avg_ror, long_investment, short_avg_ror, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
                # Total investment
                total_investment = long_investment + short_investment

                long_returns, short_returns, both_returns = get_actual_returns(y_pred_binary, lastPrice, futurePrice, prediction)

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Returns' : long_returns,
                    'Long_RoR': long_returns/long_investment,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment, 
                    'Short_Returns' : short_returns,
                    'Short_RoR': short_returns/short_investment,
                    'Total_Investment': total_investment,
                    'Total_Returns': both_returns,
                    'Total_RoR': both_returns/total_investment
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/separate/Silver_rolling_transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/SmallCap_rolling_tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/separate/rolling_MSCI_test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/rolling_MSCI_test_evaluation_scores.csv'.")

Evaluating model predicting 1 days ahead using MPA images with 7 days timeframe.
29
1/1 [==============================] - 0s 247ms/step
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.8408 - accuracy: 0.6522 - val_loss: 3.5361 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 227ms/step - loss: 0.9997 - accuracy: 0.8261 - val_loss: 3.0521 - val_accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 227ms/step - loss: 0.4003 - accuracy: 0.9130 - val_loss: 2.4074 - val_accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 0s 223ms/step - loss: 0.1435 - accuracy: 0.9565 - val_loss: 1.8243 - val_accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 225ms/step - loss: 0.1068 - accuracy: 0.9565 - val_loss: 1.3868 - val_accuracy: 0.6667
Epoch 6/10
1/1 [==============================] - 0s 233ms/step - loss: 0.0200 - accuracy: 1.0000 - val_loss: 1.1332 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==========

In [ ]:
# Overlay our three new images and evaluate the combined models on the newly crated dataset using rolling windows

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 
                                      'Long_Average_RoR', 'Long_Investment', 'Long_Returns', 'Long_RoR', 
                                      'Short_Average_RoR', 'Short_Investment', 'Short_Returns', 'Short_RoR', 
                                      'Total_Investment', 'Total_Returns', 'Total_RoR'])

TIMEFRAMES = [7, 14, 30]
PREDICTIONS = [1, 5]

IMG_TYPES = ['CPA', 'MLC', 'API']

if TRANSFER:
    etf_names = ["SmallCap"]#, "Treasury", "Silver", "Semiconductor"]
else:
    #etf_names = ["S&P500", "MSCIWorld", "DAX", "EmergingMarkets"]
    etf_names = ["MSCIWorld"]

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Evaluating model predicting {prediction} days ahead using overelayed images with {timeframe} days timeframe.")

            # Filter the data
            data = labels[(labels['TimePrediction'] == prediction) &
                            (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)

            # Build df with overlayed images
            overlayed_df = pd.DataFrame(columns=['Image_Array', 'LastPrice', 'FuturePrice', 'Label'])

            # For each timeframe, prediction and ETF overlay the images
            for date in data['Date'].unique():
                try:
                    # Filter the data
                    data_date = data[(data['Date'] == date)]
                    # Load the images
                    img1 = data_date[data_date['Image'].str.contains('CPA')]['Image_Array'].values[0]
                    img2 = data_date[data_date['Image'].str.contains('MLC')]['Image_Array'].values[0]
                    img3 = data_date[data_date['Image'].str.contains('API')]['Image_Array'].values[0]
                    # Overlay the images
                    img = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)
                    img = cv2.addWeighted(img, 0.5, img3, 0.5, 0)

                    # Save image to file
                    cv2.imwrite(f'images/overlayed/{etf}_{date}_{prediction}_{timeframe}.png', img)

                    # Add the image to the DataFrame
                    overlayed_df = overlayed_df.append({
                        'Image_Array': img,
                        'LastPrice': data_date['LastPrice'].values[0],
                        'FuturePrice': data_date['FuturePrice'].values[0],
                        'Label': data_date['Label'].values[0]
                    }, ignore_index=True)
                except:
                    pass
            overlayed_df = overlayed_df.reset_index(drop=True)

            # Load the model
            model_filename = f"models/MSCIWorld_combined_{timeframe}_{prediction}.h5"
            model = load_model(model_filename)

            X = np.array(overlayed_df['Image_Array'].tolist()) / 255.0
            lastPrice = overlayed_df['LastPrice'].tolist()
            futurePrice = overlayed_df['FuturePrice'].tolist()
            y = overlayed_df['Label'].values
            y = np.array(y, dtype=np.int32)

            # Initialize y_pred_binary as an empty array
            y_pred_binary = np.array([], dtype=int)
            
            # Create a rolling window of 10
            window_size = len(X)/12
            print(int(window_size))

            for window in range(12):
                if window == 0:
                    window_X = X[0:int(window_size)]
                    window_lastPrice = lastPrice[0:int(window_size)]
                    window_futurePrice = futurePrice[0:int(window_size)]
                    window_y = y[0:int(window_size)]

                    # Evaluate the model on test data
                    y_pred_window = model.predict(window_X)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                    y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])
                elif window == 11:
                    model.fit(window_X, window_y, epochs=10, batch_size=32)
                    window_X = X[int(window_size*window):]
                    window_lastPrice = lastPrice[int(window_size*window):]
                    window_futurePrice = futurePrice[int(window_size*window):]
                    window_y = y[int(window_size*window):]

                    # Evaluate the model on test data
                    y_pred_window = model.predict(window_X)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                    y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])
                else:
                    model.fit(window_X, window_y, epochs=10, batch_size=32)
                    window_X = X[int(window_size*window):int(window_size*(window+1))]
                    window_lastPrice = lastPrice[int(window_size*window):int(window_size*(window+1))]
                    window_futurePrice = futurePrice[int(window_size*window):int(window_size*(window+1))]
                    window_y = y[int(window_size*window):int(window_size*(window+1))]

                    # Evaluate the model on test data
                    y_pred_window = model.predict(window_X)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary_window = np.where(y_pred_window > 0.5, 1, 0).flatten()
                    y_pred_binary = np.concatenate([y_pred_binary, y_pred_binary_window])       

            
            accuracy = accuracy_score(y, y_pred_binary)
            precision = precision_score(y, y_pred_binary)
            recall = recall_score(y, y_pred_binary)
            f1_score = fbeta_score(y, y_pred_binary, beta=1)

            y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            long_avg_ror, long_investment, short_avg_ror, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
            # Total investment
            total_investment = long_investment + short_investment

            long_returns, short_returns, both_returns = get_actual_returns(y_pred_binary, lastPrice, futurePrice, prediction)
            #total_returns = long_returns + short_returns
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Long_Average_RoR': long_avg_ror,
                'Long_Investment': long_investment,
                'Long_Returns' : long_returns,
                'Long_RoR': long_returns/long_investment,
                'Short_Average_RoR': short_avg_ror,
                'Short_Investment': short_investment, 
                'Short_Returns' : short_returns,
                'Short_RoR': short_returns/short_investment,
                'Total_Investment': total_investment,
                'Total_Returns': both_returns,
                'Total_RoR': both_returns/total_investment
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/overlayed/Treasury_transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/rolling_tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/overlayed/rolling_MSCIWorld_test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/rolling_MSCIWorld_test_evaluation_scores.csv'.")